In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.0     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.1.0
✔ tidyr   1.2.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 1.0.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
# input parameters

blockfsum_file <- "/mnt/archgen/public_data/HGDP+1KG_callset/stats/POS.all_filtered.AFR_mono.5EUR/f3.Out.1000G-EUR.HGDP-EUR.EUR_asc_min104/fsum/AFR.FIN.HGDP00511.txt"

block_id_all <- 1:22

min_freq_all <- 1

max_freq_all <- 1:100

all_sites_block <- TRUE

min_sites_block <- 50  # Apply only when processing by different frequencies; suitable for samples that have different covered rate among different frequencies (capture data). Default: NA.

#min_sites_block <- NA

In [2]:
# input parameters

blockfsum_file <- "/mnt/archgen/users/lei_huang/RAS_tools/1.txt"

block_id_all <- 1:22

min_freq_all <- 0

max_freq_all <- 0:60

all_sites_block <- TRUE

min_sites_block <- 1000  # Apply only when processing by different frequencies; suitable for samples that have different covered rate among different frequencies (capture data). Default: NA.

#min_sites_block <- NA

In [3]:
min_freq_min <- min(min_freq_all)
max_freq_max <- max(max_freq_all)
asc_conds <- data.frame(min_freq=min_freq_all, max_freq=max_freq_all)

In [4]:
fsum_block_asc_cutoff <- utils::read.table(blockfsum_file, header = TRUE, sep = "\t")

In [6]:
fsum_block_asc_cutoff[fsum_block_asc_cutoff$asc_freq_max==100, "Nr_sites_all"] - fsum_block_asc_cutoff[fsum_block_asc_cutoff$asc_freq_max==64, "Nr_sites_all"]

[1] 294 455 327 231 164 266 266 195 212 194 233 179  99 142 186 114 108 112  87
[20] 145 126  60

In [7]:
fsum_block_asc_cutoff[fsum_block_asc_cutoff$asc_freq_max==100, c("Nr_sites_all", "Nr_sites_nonmissing", "fsum")] - fsum_block_asc_cutoff[fsum_block_asc_cutoff$asc_freq_max==64, c("Nr_sites_all", "Nr_sites_nonmissing", "fsum")]

,Nr_sites_all,Nr_sites_nonmissing,fsum
,<int>,<int>,<dbl>
100,294,222,0.9701355
383,455,394,1.4494742
666,327,295,2.7472766
949,231,196,1.1166308
1232,164,121,0.4489794
1515,266,238,2.0222706
1798,266,214,1.0921387
2081,195,149,0.7536933
2364,212,135,1.0182378


In [5]:
f_block_all <- data.frame()

In [13]:
# only for all_sites_block=TRUE and !is.na(min_sites_block)
# divide the cutoff so that in each cutoff there is similar proportion of non-missing SNPs among different frequency.

cutoff_range <- list()
asc_freq_all <- base::unique(fsum_block_asc_cutoff[, "asc_freq_max"]) %>% base::sort()

for (asc_max in base::rev(asc_freq_all[asc_freq_all <= max_freq_max]))
{
    #print(asc_max)
    cutoff_range[[as.character(asc_max)]] <- list()

    asc_min <- asc_max
    while (asc_min > min_freq_min)
    {
        Nr_sites_all <- fsum_block_asc_cutoff[fsum_block_asc_cutoff$asc_freq_max==asc_max, "Nr_sites_all"] - fsum_block_asc_cutoff[fsum_block_asc_cutoff$asc_freq_max==asc_min-1, "Nr_sites_all"]
        if (all(Nr_sites_all >= min_sites_block))
            break
        asc_min <- asc_min - 1
    }
    if (asc_min > min_freq_min)
    {
        f_block <- fsum_block_asc_cutoff[fsum_block_asc_cutoff$asc_freq_max==asc_max, c("Nr_sites_all", "Nr_sites_nonmissing", "fsum")] - fsum_block_asc_cutoff[fsum_block_asc_cutoff$asc_freq_max==asc_min-1, c("Nr_sites_all", "Nr_sites_nonmissing", "fsum")]
    } else 
        f_block <- fsum_block_asc_cutoff[fsum_block_asc_cutoff$asc_freq_max==asc_max, c("Nr_sites_all", "Nr_sites_nonmissing", "fsum")]
    f_block <- f_block %>% mutate(f=fsum/Nr_sites_nonmissing)

    cutoff_range[[as.character(asc_max)]]$asc_min <- asc_min
    cutoff_range[[as.character(asc_max)]]$f_block <- f_block
}

In [ ]:
cutoff_range[[1]]

,Nr_sites_all,Nr_sites_nonmissing,fsum,f
,<int>,<int>,<dbl>,<dbl>
1,818994,805794,0.122192530,1.516424e-07
284,884832,872563,0.105735075,1.211776e-07
567,725793,716978,0.089478370,1.247993e-07
850,722685,713694,0.091898850,1.287651e-07
1133,667057,658613,0.110016405,1.670426e-07
1416,611911,604444,0.061696945,1.020722e-07
1699,593025,583561,0.067297250,1.153217e-07
1982,568278,560559,0.041547695,7.411833e-08
2265,459775,451186,0.043704755,9.686638e-08


In [ ]:
cutoff_range[[64]]

,Nr_sites_all,Nr_sites_nonmissing,fsum,f
,<dbl>,<dbl>,<dbl>,<dbl>
64,339,292,0.93595275,0.003205318
164,455,415,0.72958360,0.001758033
264,306,276,0.58163265,0.002107365
364,268,225,0.72856120,0.003238050
464,252,222,0.66810345,0.003009475
564,287,275,1.09751775,0.003990974
664,225,188,0.46160210,0.002455330
764,219,180,0.72500625,0.004027813
864,259,217,0.32666205,0.001505355


In [ ]:
cutoff_range[[100]]

,Nr_sites_all,Nr_sites_nonmissing,fsum,f
,<dbl>,<dbl>,<dbl>,<dbl>
100,261,193,0.9522784,0.004934085
200,402,350,1.3117190,0.003747769
300,287,257,2.5916644,0.010084297
400,205,174,1.0196920,0.005860299
500,142,107,0.2576529,0.002407971
600,233,206,1.8156379,0.008813776
700,213,166,1.0028531,0.006041283
800,170,128,0.7536933,0.005888229
900,171,99,0.5309931,0.005363567


In [14]:
for (i in 1:nrow(asc_conds))
{
    min_freq <- asc_conds$min_freq[i]
    max_freq <- asc_conds$max_freq[i]
    
    asc_max <- max_freq
    fsum <- 0
    Nr_sites_all <- 0
    repeat
    {
        f_block_cutoff <- cutoff_range[[as.character(asc_max)]]$f_block
        fsum <- fsum + f_block_cutoff$Nr_sites_all * f_block_cutoff$f
        Nr_sites_all <- Nr_sites_all + f_block_cutoff$Nr_sites_all
        
        asc_max <- cutoff_range[[as.character(asc_max)]]$asc_min - 1
        if (asc_max < min_freq)
            break
    }
    f_block_all <- f_block_all %>% rbind(data.frame(block_id=block_id_all, jackknife_weight=Nr_sites_all, f=fsum/Nr_sites_all, Asc_min=min_freq, Asc_max=max_freq))
}

In [15]:
f_block_all

block_id,jackknife_weight,f,Asc_min,Asc_max
<int>,<dbl>,<dbl>,<dbl>,<int>
1,2113602,1.874785e-07,0,0
2,2282812,1.727222e-07,0,0
3,1880162,0.000000e+00,0,0
4,1841907,0.000000e+00,0,0
5,1707403,7.172848e-07,0,0
6,1577294,2.507268e-07,0,0
7,1518910,0.000000e+00,0,0
8,1469911,0.000000e+00,0,0
9,1174893,3.536698e-07,0,0


In [ ]:
# all_sites_block & is.na(min_sites_block) | !all_sites_block

for (i in 1:nrow(asc_conds))
{
    min_freq <- asc_conds$min_freq[i]
    max_freq <- asc_conds$max_freq[i]
    
    if (min_freq > min_freq_min)
    {
        f_block <- fsum_block_asc_cutoff[fsum_block_asc_cutoff$asc_freq_max==max_freq, c("Nr_sites_all", "Nr_sites_nonmissing", "fsum")] - fsum_block_asc_cutoff[fsum_block_asc_cutoff$asc_freq_max==min_freq-1, c("Nr_sites_all", "Nr_sites_nonmissing", "fsum")]
    } else 
        f_block <- fsum_block_asc_cutoff[fsum_block_asc_cutoff$asc_freq_max==max_freq, c("Nr_sites_all", "Nr_sites_nonmissing", "fsum")]

    f_block_all <- f_block_all %>% rbind(data.frame(block_id=block_id_all, jackknife_weight=if (all_sites_block) f_block$Nr_sites_all else f_block$Nr_sites_nonmissing, f=f_block$fsum/f_block$Nr_sites_nonmissing, Asc_min=min_freq, Asc_max=max_freq))
}

In [ ]:
# no ascertainment

fsum_block_asc_cutoff %>% mutate(jackknife_weight=if (all_sites_block) Nr_sites_all else Nr_sites_nonmissing, f=fsum/Nr_sites_nonmissing) %>% select(block_id, jackknife_weight, f)

,block_id,jackknife_weight,f
,<int>,<int>,<dbl>
1,1,1164619,1.587945e-05
2,2,1253560,1.851479e-05
3,3,1018452,2.061655e-05
4,4,1012862,1.819510e-05
5,5,934529,1.590488e-05
6,6,864837,2.199142e-05
7,7,840704,1.829802e-05
8,8,803962,1.500694e-05
9,9,659526,1.739663e-05
